In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from numpy.linalg import inv, pinv
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.svm import SVC

In [8]:
week1 = pd.read_csv("/home/vxofi/Datasets/Intrusions/week1/week1.csv", sep=";", header=None)
week2 = pd.read_csv("/home/vxofi/Datasets/Intrusions/week2/week2.csv", sep=";", header=None)
week3 = pd.read_csv("/home/vxofi/Datasets/Intrusions/week3/week3.csv", sep=";", header=None)
week4 = pd.read_csv("/home/vxofi/Datasets/Intrusions/week4/week4.csv", sep=";", header=None)
week5 = pd.read_csv("/home/vxofi/Datasets/Intrusions/week5/week5.csv", sep=";", header=None)
week6 = pd.read_csv("/home/vxofi/Datasets/Intrusions/week6/week6.csv", sep=";", header=None)
week7 = pd.read_csv("/home/vxofi/Datasets/Intrusions/week7/week7.csv", sep=";", header=None)
week8_true = pd.read_csv("/home/vxofi/Datasets/Intrusions/week8_true/week8_true.csv", sep=";", header=None)
week9_true = pd.read_csv("/home/vxofi/Datasets/Intrusions/week9_true/week9_true.csv", sep=";", header=None)
#FIX first row appears as titles

In [9]:
week1.head()

,0,1,2,3,4,5,6,7,8,9,10
0,1,06.02.1998,00:00:07,00:00:01,http,2127,80,172.016.114.207,152.163.214.011,0,-
1,2,06.02.1998,00:00:07,00:00:01,http,2139,80,172.016.114.207,152.163.212.172,0,-
2,3,06.02.1998,00:00:07,00:00:01,http,2128,80,172.016.114.207,152.163.214.011,0,-
3,4,06.02.1998,00:00:07,00:00:01,http,2129,80,172.016.114.207,152.163.214.011,0,-
4,5,06.02.1998,00:00:07,00:00:01,http,2130,80,172.016.114.207,152.163.214.011,0,-


In [15]:
train = pd.concat([week1, week2, week3, week4, week5, week6, week7], ignore_index=True)
test = pd.concat([week8_true, week9_true], ignore_index=True)

train.drop(columns=train.columns[0], axis=1,  inplace=True)
test.drop(columns=test.columns[0], axis=1,  inplace=True)

In [19]:
labels = ["Start Date", "Start Time", "Duration", "Serv", "Src Port", "Dest Port", "Src IP", "Dest IP", "Attack Score", "Name"]
train.columns = labels
test.columns = labels

In [20]:
test.head()

,Start Date,Start Time,Duration,Serv,Src Port,Dest Port,Src IP,Dest IP,Attack Score,Name
0,07/20/1998,08:00:01,00:00:01,domain/u,1114,53,192.168.001.010,172.016.112.020,0,-
1,07/20/1998,08:00:01,00:00:01,domain/u,1059,53,192.168.001.010,172.016.112.020,0,-
2,07/20/1998,08:00:31,00:00:01,snmp/u,1195,161,194.027.251.021,192.168.001.001,0,-
3,07/20/1998,08:00:31,00:00:01,urp/i,-,-,192.168.001.001,194.027.251.021,0,-
4,07/20/1998,08:00:36,00:00:01,snmp/u,1197,161,194.027.251.021,192.168.001.001,0,-


In [ ]:
stdSeq = float(np.std(df[['value']]))
meanSeq = float(np.mean(df['value']))
print(f'std: {stdSeq}, mean {meanSeq}')
df['z-scaled'] = StandardScaler().fit_transform(df[['value']])
df.head(5)

In [ ]:
class OSELM:
    def __init__(self, nInputs, nOutputs, numHiddenNeurons, forgettingFactor=1, gamma = 0.0001):

        self.numHiddenNeurons = numHiddenNeurons
        self.lamb = forgettingFactor

        # Веса и смещение выбираются случайным образом в интервале [-1; 1]
        self.alpha = np.random.random((self.numHiddenNeurons, nInputs)) * 2 - 1 
        self.bias = np.random.random((1, self.numHiddenNeurons)) * 2 - 1

        self.beta = np.zeros((self.numHiddenNeurons, nOutputs)) # Формула 9
        self.P = inv(gamma*np.eye(self.numHiddenNeurons)) # Формула 9


    def compute_h(self, features):       
        return self.sigmoid(features @ self.alpha.T  + self.bias)


    def sigmoid(self, features):
        return 1.0 / (1.0 + np.exp(-features))


    def fit(self, features, targets):
        features = np.array(features)
        targets = np.array(targets)
    
        H = self.compute_h(features)

        k = 1/self.lamb
        self.P = k * self.P - self.P @ H.T @ pinv(self.lamb**2 + self.lamb * H @ self.P @ H.T) @ H @ self.P # Формула 12

        self.beta = self.beta + self.P @ H.T @ (targets - H @ self.beta) # Формула 10
  
    def predict(self, features):
        H = self.compute_h(features)
        prediction = H @ self.beta # Формула 2
        return prediction

In [ ]:
predictions = []
targets = Y[:-1]
oselm = OSELM(100, 1, 100, forgettingFactor=0.996)
for i in range(len(df)-101):
    oselm.fit(X[i], Y[i])
    prediction = oselm.predict(X[i+1])

    predictions.append(prediction[0])